In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, AveragePooling2D
from tensorflow.keras.activations import relu, elu
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# Загружаем данные.
train = pd.read_csv('../input/digit-recognizer/train.csv')
subm = pd.read_csv('../input/digit-recognizer/test.csv')

In [3]:
# Разделяем датасет на 3-и подвыборки.
x_train, x_valid, y_train, y_valid = train_test_split(train.iloc[:, 1:], train.iloc[:, 0], test_size=0.2, shuffle=True)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)

In [4]:
# Создаем одну базу данных для тренировки выбранной модели, нормализуем и изменяем размерность.
all_data = np.apply_along_axis(lambda x: x.reshape(28, 28, 1), 1,
                               (train.iloc[:, 1:]/ 255).astype(float).to_numpy())
all_labels = train.iloc[:, 0].to_numpy() # Отделяем метки классов от датасета и переводим в np.
all_data = np.where(all_data > 0.3, all_data, 0) # Удаляем нечёткие приксели.
all_labels = utils.to_categorical(all_labels, num_classes=10) #  Переводим в One Hot E.

In [5]:
# Проверим размерность.
print(x_train.shape) 
print(x_valid.shape) 
print(x_test.shape)
print(y_train.shape) 
print(y_valid.shape)
print(y_test.shape)

In [6]:
# Нормализуем данные [0; 1] и преобразуем в numpy массив.
x_train = np.apply_along_axis(lambda x: x.reshape(28, 28, 1), 1, (x_train/ 255).astype(float).to_numpy())
x_valid = np.apply_along_axis(lambda x: x.reshape(28, 28, 1), 1, (x_valid/ 255).astype(float).to_numpy())
x_test = np.apply_along_axis(lambda x: x.reshape(28, 28, 1), 1, (x_test/ 255).astype(float).to_numpy())
subm = np.apply_along_axis(lambda x: x.reshape(28, 28, 1), 1, (subm/ 255).astype(float).to_numpy())
y_train = y_train.to_numpy()
y_valid = y_valid.to_numpy()
y_test = y_test.to_numpy()

In [7]:
# Проверим полученную размерность.
print(x_train.shape) 
print(x_valid.shape)
print(x_test.shape) 
print(y_train.shape) 
print(y_valid.shape)
print(y_test.shape)
print(subm.shape)

In [8]:
# Переведём метки в OHE.
y_train = utils.to_categorical(y_train, num_classes=10)
y_valid = utils.to_categorical(y_valid, num_classes=10)
y_test = utils.to_categorical(y_test, num_classes=10)

In [9]:
# Отрисовка примеров из датасета.
fig = plt.figure(figsize=(8, 6))
fig.suptitle('Примеры цифр из датасета.', fontsize=20)
for i in range(1, 10):
    plt.subplot(3, 3, i)
    plt.imshow(x_train[i].reshape(28, 28))
    plt.xticks([])
    plt.yticks([])

In [22]:
# Создадим модель.
inputs = keras.Input(shape=(28, 28, 1))
x = Conv2D(64, 3)(inputs)
x = BatchNormalization()(x)
x = relu(x)
x = Conv2D(64, 3)(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv2D(64, 5, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = relu(x)
x = Dropout(0.4)(x)
x = Conv2D(128, 3)(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv2D(128, 3)(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv2D(128, 5, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = relu(x)
x = Dropout(0.4)(x)
x = Flatten()(x)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = relu(x)
x = Dropout(0.4)(x)
outputs = Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [23]:
# Статистика по модели.
model.summary()

In [24]:
# Компиляция модели.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
# Создание понижения уровня обучения и сохранения лучшей модели в течении обучения.
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                                 patience=5, min_lr=0.00000001, verbose=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='model.hdf5', monitor='val_loss',
                                                save_best_only=True, verbose=1)

In [25]:
# Для всех данных.
# Создание понижения уровня обучения и сохранения лучшей модели в течении обучения.
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1,
                                                 patience=5, min_lr=0.0000000001, verbose=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='model.hdf5', monitor='loss',
                                                save_best_only=True, verbose=1)

In [26]:
# Генератор изображений на основе имеющихся, для расширения кол-ва образцов(аугментация).
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [ ]:
# Тренировка модели на разбитых ранее данных.
history = model.fit(datagen.flow(x_train, y_train, batch_size=64),
                    validation_data=datagen.flow(x_valid, y_valid),
                    callbacks=[reduce_lr,checkpoint], epochs=100, verbose=1)

In [ ]:
# Оценка модели.
model.evaluate(x_test, y_test)

In [27]:
# Обучение мродели на всех данных.
history = model.fit(datagen.flow(all_data, all_labels, batch_size=64),
                    callbacks=[reduce_lr,checkpoint], epochs=100, verbose=1)

In [31]:
# Загружаем лучшую модель.
l_model = tf.keras.models.load_model('./model.hdf5')

In [32]:
# Классификация тестовой выборки с переводом OHE в метки.
y_pred = model.predict(subm)
y = []
for i in range(0,len(y_pred)):
    prob = y.append(np.argmax(list(y_pred[i])))

np.unique(y)

In [36]:
# Перевод в датафрейм полученной классификации.
submission = pd.DataFrame({"ImageId":range(1,len(y_pred)+1),"Label":y})
submission.head()

In [37]:
# Сохранение полученного датафрейма.
submission = submission.to_csv('./submission.csv', index=False, header=True)# , encoding='utf-8'